# Garbage Collection

In [34]:
import torch as th

In [35]:
import syft as sy

In [36]:
hook = sy.TorchHook(th)

In [37]:
bob = sy.VirtualWorker(hook,id="bob")

In [38]:
x = th.tensor([1,2,3,4]).send(bob)

In [39]:
bob._objects

{34159474489: tensor([1, 2, 3, 4])}

In [40]:
del x

In [41]:
bob._objects

{}

In [42]:
for i in range(1000):
    x = th.tensor([1,2,3,4]).send(bob)

In [43]:
bob._objects

{60849088953: tensor([1, 2, 3, 4])}

In [44]:
del x

In [45]:
bob._objects

{}

# Toy Federated Learning

In [46]:
from torch import nn, optim

In [47]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [48]:
model = nn.Linear(2,1)

In [49]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [50]:
def training(iterations):
    for iter in range(iterations):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
training(20)

tensor(3.3743)
tensor(1.0765)
tensor(0.6462)
tensor(0.4400)
tensor(0.3063)
tensor(0.2152)
tensor(0.1525)
tensor(0.1088)
tensor(0.0783)
tensor(0.0567)
tensor(0.0413)
tensor(0.0303)
tensor(0.0223)
tensor(0.0165)
tensor(0.0123)
tensor(0.0092)
tensor(0.0069)
tensor(0.0052)
tensor(0.0039)
tensor(0.0029)


In [51]:
naufil = sy.VirtualWorker(hook, id="naufil")
faizan = sy.VirtualWorker(hook, id="faizan")

In [52]:
data_naufil = data[0:2].send(naufil)
target_naufil = target[0:2].send(naufil)

In [53]:
data_faizan = data[2:4].send(faizan)
target_faizan = target[2:4].send(faizan)

In [54]:
datasets = [(data_naufil,target_naufil),(data_faizan,target_faizan)]

In [55]:
model = nn.Linear(2,1)

In [56]:
opt = optim.SGD(params = model.parameters(), lr = 0.1)

In [57]:
data1, target1 = datasets[0]

In [58]:
data1.location

<VirtualWorker id:naufil #tensors:6>

In [61]:
target1.location

<VirtualWorker id:naufil #tensors:7>

In [62]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0510, -0.0615]], requires_grad=True), Parameter containing:
 tensor([-0.0658], requires_grad=True)]

In [63]:
model = model.send(data1.location)

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()

# Advanced Remote Execution

In [63]:
naufil.clear_objects
faizan.clear_objects

<bound method BaseWorker.clear_objects of <VirtualWorker id:faizan #tensors:4>>

In [53]:
x = th.tensor([1,2,3,4]).send(naufil).send(faizan)

In [54]:
x

(Wrapper)>[PointerTensor | me:92732758512 -> faizan:75315879446]

In [55]:
faizan._objects

{23613880314: (Wrapper)>[PointerTensor | faizan:23613880314 -> naufil:83904550070],
 47735554887: tensor([[1, 0],
         [0, 0]]),
 76176561640: tensor([[1, 0],
         [0, 0]]),
 75315879446: (Wrapper)>[PointerTensor | faizan:75315879446 -> naufil:67688684810]}

In [56]:
naufil._objects

{83904550070: tensor([1, 2, 3, 4]), 49095176888: tensor([[1, 1],
         [0, 1]]), 36408048950: tensor([[1, 1],
         [0, 1]]), 67688684810: tensor([1, 2, 3, 4])}

In [57]:
del x

In [58]:
naufil._objects

{83904550070: tensor([1, 2, 3, 4]), 49095176888: tensor([[1, 1],
         [0, 1]]), 36408048950: tensor([[1, 1],
         [0, 1]]), 67688684810: tensor([1, 2, 3, 4])}

In [59]:
faizan._objects

{23613880314: (Wrapper)>[PointerTensor | faizan:23613880314 -> naufil:83904550070],
 47735554887: tensor([[1, 0],
         [0, 0]]),
 76176561640: tensor([[1, 0],
         [0, 0]]),
 75315879446: (Wrapper)>[PointerTensor | faizan:75315879446 -> naufil:67688684810]}

In [21]:
x

NameError: name 'x' is not defined